# Getting __more__ stock data

In [2]:
import numpy as np
import pandas as pd
import dill
from sqlalchemy import create_engine, inspect
from tqdm import tqdm_notebook
from alpha_vantage.timeseries import TimeSeries

So I'm going to try another API provider, [Alpha Vantage](https://www.alphavantage.co/documentation/), to see if they offer a more complete listing of end of day stock prices. The number to beat is 110 companies.

...Lets get started by importing our set of tickers

In [3]:
future_Ticks = dill.load(open('future_ticker_symbols.pkl', 'r'))

In [4]:
past_Ticks = dill.load(open('past_ticker_symbols.pkl', 'r'))

In [5]:
tickSymbs = future_Ticks.union(past_Ticks)

...and our Alpha Vantage API key, from an ignored file not pushed to my public repo

In [6]:
av_key_handle = open("alphavantage.apikey", "r")
ts = TimeSeries(key=av_key_handle.read().strip(), output_format='pandas')
av_key_handle.close()

...finally we can connect to the sqlite database and start pulling down data

In [7]:
engine = create_engine('sqlite:///training_data.db')

In [8]:
losers = []
winners = []
for ticker in tqdm_notebook(tickSymbs):
    try:
        df, meta = ts.get_daily(symbol=ticker, outputsize='full')
        df.to_sql(meta["2. Symbol"], engine, if_exists = 'replace')
    except:
        losers.append(ticker)
    else:
        winners.append(meta["2. Symbol"])

In [9]:
print len(winners), winners
print len(losers), losers

273 ['AGN', 'LIFE', 'AAAP', 'SAGE', 'BPTH', 'AGEN', 'ESPR', 'RETA', 'IRWD', 'VNDA', 'BYSI', 'CNAT', 'ALDR', 'VRTX', 'AGLE', 'DRRX', 'EXEL', 'FLXN', 'XBIT', 'JNCE', 'MEIP', 'ADMP', 'ADMS', 'OTIC', 'ACOR', 'CEMP', 'KMPH', 'ANAB', 'TNXP', 'ITEK', 'JNJ', 'PSTI', 'SNY', 'AMAG', 'AFMD', 'GWPH', 'GBT', 'CRMD', 'LPCN', 'SVRA', 'VCEL', 'PTGX', 'ADXS', 'PHMMF', 'DCTH', 'TBPH', 'DVAX', 'SGEN', 'IOVA', 'BHVN', 'NVLS', 'EGRX', 'AXSM', 'GLMD', 'CPRX', 'DBVT', 'RDUS', 'NVS', 'RNN', 'ALDX', 'OPHT', 'KURA', 'CBIO', 'NVO', 'OPK', 'EBIO', 'HRTX', 'ONTX', 'MRUS', 'SPPI', 'AMGN', 'CRBP', 'KTOV', 'SMMT', 'AZN', 'VRX', 'HALO', 'MBVX', 'BLRX', 'LOXO', 'SNDX', 'CNCE', 'IMGN', 'LJPC', 'MNOV', 'GALE', 'ATRS', 'CLSD', 'INNL', 'CLSN', 'TXMD', 'CHRS', 'PLX', 'EARS', 'AKBA', 'MYOK', 'OCRX', 'EPZM', 'PTCT', 'CYCC', 'FLKS', 'BLPH', 'ATNM', 'MRNS', 'HSGX', 'PGNX', 'TENX', 'MDCO', 'IMRN', 'CELG', 'ATNX', 'RXII', 'VTGN', 'APRI', 'AERI', 'NEOS', 'RDHL', 'CYTK', 'MESO', 'OCUL', 'ICPT', 'DMTX', 'ADRO', 'ACRS', 'CERC', 'ACRX

Lets check in the morning how she did, how far back can she read and what % of companies were successfully pulled?

In [14]:
df

,volume,close,high,open,low
2013-07-15,600.0,7.0500,7.0500,7.0000,7.0000
2013-07-16,1400.0,8.0100,8.0100,7.5000,7.5000
2013-07-17,2000.0,8.5000,8.7500,8.5000,8.2000
2013-07-18,7000.0,9.2000,11.7900,10.8800,9.1800
2013-07-19,1400.0,9.0000,9.2000,9.2000,9.0000
2013-07-22,1000.0,10.0000,10.0000,9.9700,9.5000
2013-07-23,700.0,9.5000,9.5000,9.0225,9.0225
2013-07-24,20400.0,9.0100,10.0300,10.0000,9.0100
2013-07-25,4200.0,9.0000,9.0500,9.0000,8.9990
2013-07-26,300.0,8.5000,8.8400,8.8400,8.5000
